<a href="https://colab.research.google.com/github/rodrigomariamorgao/portfolio_data_science/blob/master/Detec%C3%A7%C3%A3o_de_Fraude_em_Cart%C3%B5es_de_Cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="BannerDataScience" width="100%" src="https://raw.githubusercontent.com/rodrigomariamorgao/portfolio_data_science/master/banner.png">

In [ ]:
#@title
# suprimir os warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# **Detecção de Fraudes em Cartões de Crédito**

Neste projeto iremos abordar o problema das fraudes em cartões de crédito, uma das principais preocupações das instituições financeiras como bancos e *fintechs*. Apenas no Brasil cerca de **12,1 milhões de pessoas** já foram vítimas de algum tipo de fraude financeira no último ano. Traduzindo em valores, os golpes financeiros ultrapassaram a cifra de **R$ 1,8 bilhão de prejuízo** para os últimos 12 meses.

<p align=center>
<img src="https://www.creditooudebito.com/wp-content/uploads/2019/04/fraude-cartao.jpg" width="60%"></p>

Dentre essas fraudes, as que envolvem cartões de crédito são de grande relevância, uma vez que a sua não-detecção acaretará em prejuízos consideráveis, tanto para o consumidor quanto para a instituição financeira.

Um outro fator a ser considerado é a quantidade de falsos positivos, ou seja, aquelas vezes em que você tentou fazer uma compra e teve seu cartão bloqueado preventivamente - o que provavelmente gerou estresse e constrangimento.

Por todos esses motivos, o investimento na área de detecção de fraudes por meio de Inteligência Artificial vem crescendo a cada ano, representando uma grande oportunidade em *Data Science*. 

Dispondo de grandes volumes de dados como base histórica, um algoritmo de machine learning apenas um pouco melhor que os anteriores já representa uma **economia de milhões de Reais**. E esse é o desafio: aprimorar cada vez mais o uso de algoritmos visando inibir ou evitar transações fraudulentas.

## **Importando os Dados**

Os dados que usaremos neste projeto foram disponibilizados por algumas empresas européias de cartão de crédito. O *dataset* representa as operações financeiras que aconteceram no período de dois dias, onde foram classificadas 492 fraudes em meio a quase 290 mil transações.

Como você pode notar, este é um conjunto de dados extremamente desbalanceado, onde as fraudes representam **apenas 0,17% do total**.

Outro detalhe interessante é que as *features* são todas numéricas, pois foram descaracterizadas (por restrições ligadas à privacidade e segurança). Assim, os nomes das colunas são representados por $[V1, V2, V3 \dots, V28]$. 

<p align=center>
<img src="https://image.freepik.com/free-vector/landing-page-concept-credit-card-payment_23-2148303916.jpg" width="50%"></p>

[Na página original dos dados](https://www.kaggle.com/mlg-ulb/creditcardfraud) também é informado que as variáveis passaram por uma transformação conhecida como Análise de Componentes Principais (*Principal Component Analysis* - PCA).

A PCA permite a redução da dimensionalidade enquanto mantém o maior número possível de informações. Para conseguir isso, o algoritmo encontra um conjunto novo de recursos - os chamados **componentes**.

Esses componentes são em número menor ou igual às variáveis originais. No caso deste projeto, os componentes resultantes pela transformação da PCA são as próprias colunas $[V1, V2, V3 \dots, V28]$.

In [ ]:
# importar o pacote pandas
import pandas as pd

# importar o pacote numpy
import numpy as np

# definir que o resultado aleatório sempre seja o mesmo
np.random.seed(123)

# caminho do dataset
file_path = "https://www.dropbox.com/s/hlx9qxh02zn228o/creditcard.csv?dl=1"

# importar os dados para um dataframe
df = pd.read_csv(file_path)

Com os dados importados para dentro de uma estrutura *Dataframe* - e não havendo a necessidade de mais nenhum ajuste ou configuração nesta etapa, pode-se iniciar uma análise exploratória dos dados a fim de preparar um modelo de *Machine Learning*.

## **Análise Exploratória**

Vamos analisar as 5 primeiras entradas de nosso *Dataframe*.

In [ ]:
# verificar as 5 primeiras entradas do Dataframe
df.head()

A seguir iremos detalhar o significado das variáveis acima.


### **Dicionário de variáveis**

As variáveis (também podemos chamá-las de colunas ou *features*) serão detalhadas abaixo, para nível de conhecimento de nosso *Dataframe*.

Esta etapa tem por objetivo demostrar a situação inicial dos dados e permitir um entendimento de como os mesmos estão estruturados.
1. `Time` - Número de segundos decorridos entre a transação da linha selecionada e a primeira transação no conjunto de dados.
1. `V1 - V28` -  Resultado da redução de dimensionalidade pela transformação PCA, para proteger dados sensíveis e identidade de usuários.
1. `Amount` - Valor da transação.
1. `Class` - Se a transação for fraudulenta, será sinalizada por 1 (um). Caso contrário, será 0 (zero).



Vamos analisar o total de variáveis e entradas que nosso *Dataframe* possui.

In [ ]:
# verificar as variáveis e entradas do Dataframe
print(f"Variáveis:\t {df.shape[1]}")
print(f"Entradas:\t {df.shape[0]}")

Veremos a seguir um resumo estatístico do *Dataframe*.

In [ ]:
# verificar o resumo do Dataframe
df.describe()

Nessa análise podemos afirmar que o valor médio das transações (coluna *Amount*) foi de **U$$ 88.34**. Sendo assim, podemos verificar que as transações não possuíam valores altos.

Em relação a coluna *Time*, podemos também afirmar que o maior tempo em relação à primeira transação foi de **172.792 segundos (48 horas de transações)**. Isso nos dá uma média de **99 transações por minuto**.

Para criarmos um modelo de Machine Learning, nosso *Dataframe* não deve conter dados nulos. Vamos verificar se existe algum campo nessa condição. 

In [ ]:
# verificar quantos dados nulos o Dataframe possui
df.isnull().sum()

Como podemos verificar, nossas variáveis não possuem nenhum campo nulo.

Vamos analisar nossa coluna alvo *Class*, onde exibe se a transação foi fraudulenta ou não.

In [ ]:
# imprimir a quantidade do tipo de transações no Dataframe
print(df.Class.value_counts())

In [ ]:
#@markdown A maioria das transações foram lícitas. Observemos a baixa porcentagem de transações fraudulentas, juntamente com um gráfico para melhor comparação visual.
print((df[df.Class == 1].shape[0] / df.shape[0]) * 100)

In [ ]:
# importar as bibliotecas matplotlib e seaborn
import matplotlib.pyplot as plt
import seaborn as sns

# plotar gráfico de barras para exibir visualmente as classes
fig, ax = plt.subplots()
sns.countplot('Class', data=df, ax=ax)
plt.text(0, 305000, 'Lícitas X Fraudes',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.tight_layout()

Vamos analisar graficamente a evolução das transações lícitas e ilícitas, para gerarmos insights em nossa avaliação.

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10,5))

x = df.Time[df.Class == 0]

ax.hist(x, bins=30)

hour_starts = []
hour_names = ["00:00"]
hour_increment = 0

for i in range(0, 175000, 10800):
  hour_increment = hour_increment + 3
  hour_starts.append(i)
  hour_names.append("{}:00".format(hour_increment))

ax.set_xticks(hour_starts)
ax.set_xticklabels(hour_names)

ax.text(0, 15000, 'Lícitas',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.xlabel('Tempo (horas)')
plt.ylabel('Transações')
plt.tight_layout()

No gráfico anterior podemos afirmar que, entre as 21 e 24h de coletas de transações em nosso *Dataset*, ocorreram o maior número de transações lícitas simultâneas.

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10,5))

x = df.Time[df.Class == 1]

ax.hist(x, bins=30)

hour_starts = []
hour_names = ["00:00"]
hour_increment = 0

for i in range(0, 175000, 10800):
  hour_increment = hour_increment + 3
  hour_starts.append(i)
  hour_names.append("{}:00".format(hour_increment))

ax.set_xticks(hour_starts)
ax.set_xticklabels(hour_names)

ax.text(0, 45, 'Fraudes',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.xlabel('Tempo (horas)')
plt.ylabel('Transações')
plt.tight_layout()

Já no gráfico a respeito das fraudes, podemos afirmar que, em torno das 12h iniciais, ocorreram o maior número de transações fraudulentas simultaneamente. Infelizmente não temos a coluna com o horário real, pois seria interessante para visualizarmos qual horário é o preferido para tentativa de fraudes.

Podemos verificar o relacionamento entre variáveis, utilizando um mapa de calor.

In [ ]:
# criar um heatmap
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(), linewidths=.2)
ax.text(0, -0.5, 'Mapa de calor',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.tight_layout()

Com esse mapa de calor podemos verificar quais variáveis possuem total relação (cor mais clara) e nenhuma relação (cor mais escura). Na tabela abaixo podemos verificar as mesmas informações, de outra maneira, quanto mais próximo de 1, melhor a relação entre as variáveis.

In [ ]:
df.corr().style.background_gradient(cmap='RdBu_r')

Nesse modo, podemos notar que a variável `Class` não possui correlação forte com as variáveis `V6, V7 e V9`, entre outras, com resultado abaixo de 0 (zero).

## **Preparação dos Dados**

Nessa etapa, iremos preparar nossos dados para que possamos construir o modelo de Machine Learning. Iniciaremos normalizando os dados das colunas `Time` e `Amount`.

In [ ]:
# importar o pacote StandardScaler
from sklearn.preprocessing import StandardScaler

# criar uma cópia do Dataframe original
df_copy = df.copy()

# instanciar o pacote para normalizar
std_scaler = StandardScaler()

# remover as colunas da cópia do Dataframe, para recriarmos no próximo passo
df_copy.drop(['Time', 'Amount'], axis=1, inplace=True)

# recriar as colunas, normalizando as mesmas
df_copy['Time'] = std_scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df_copy['Amount'] = std_scaler.fit_transform(df['Time'].values.reshape(-1, 1))

# ver as primeiras entradas do Dataframe alterado
df_copy.head()

Nosso próximo passo consiste dividir nossos dados entre treino e teste, para utilizarmos na construção de nosso modelo de classificação.

In [ ]:
# importar a função train_test_split
from sklearn.model_selection import train_test_split

# separar variáveis X e y
X = df_copy.drop('Class', axis=1)
y = df['Class']

# dividir o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Vamos verificar como ocorreu nossa divisão.

In [ ]:
print(f"Nosso Dataset contém {df_copy.shape[1]} variáveis e {df_copy.shape[0]} colunas.")
print(f"Nossos dados de treino são: X = {X_train.shape} e y = {y_train.shape}")
print(f"Nossos dados de teste são: X = {X_test.shape} e y = {y_test.shape}")

Notamos que nossos dados foram divididos de acordo com nosso parâmetro `test_size`: 80% de dados para treino e 20% de dados para teste.

## **Modelos de Machine Learning - Dados Desbalanceados**

Após a análise de nosso *Dataframe*, iremos construir modelos de classificação para validar qual terá melhor performance em nossa previsão entre transações lícitas ou fraudulentas. Os modelos escolhidos foram **Regressão Logística, Decision Trees e KNN**.

### Regressão Logística

In [ ]:
# importar o modelo LogisticRegression
from sklearn.linear_model import LogisticRegression

# instanciar o modelo
model_logistic_regression_unbalanced = LogisticRegression()

# treinamento do modelo
model_logistic_regression_unbalanced.fit(X_train, y_train)

# fazer previsões em cima de novos dados
y_pred_logistic_regression_unbalanced = model_logistic_regression_unbalanced.predict(X_test)

### Decision Trees

In [ ]:
# importar o modelo DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# instanciar o modelo e escolher os hyperparameters
model_decision_tree_unbalanced = DecisionTreeClassifier(max_depth=4, criterion="entropy")

# treinamento do modelo
model_decision_tree_unbalanced.fit(X_train, y_train)

# fazer previsões em cima de novos dados
y_pred_decision_tree_unbalanced = model_decision_tree_unbalanced.predict(X_test)

### KNN

In [ ]:
# importar o pacote neighbors
from sklearn import neighbors

# instanciar o modelo e escolher os hyperparameters
model_knn_unbalanced = neighbors.KNeighborsClassifier(n_neighbors=5)

# treinamento do modelo
model_knn_unbalanced.fit(X_train, y_train)

# fazer previsões em cima de novos dados
y_pred_knn_unbalanced = model_knn_unbalanced.predict(X_test)

## **Balanceamento dos Dados**

Lembramos que no início do projeto verificamos a proporção de atividades lícitas e fraudes, onde a mesma é muito desigual. Para  melhor performance de nosso modelo, iremos balancear o conjunto de dados.

In [ ]:
# importar a função RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler

# técnica de balanceamento Under-sampling
random_under_sampler = RandomUnderSampler()
X_random_under_sampler, y_random_under_sampler = random_under_sampler.fit_sample(X_train, y_train)

# ver como resultou o balanceamento das classes
print(pd.Series(y_random_under_sampler).value_counts())

# plotar a nova distribuição de classes
sns.countplot(y_random_under_sampler)
plt.text(-0.5, 425, 'Lícitas X Fraudes',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.tight_layout()

Agora com os dados balanceados, podemos comparar os mapas de calor pré e pós balanceamento, para verificarmos se houve alguma diferença.

In [ ]:
# criar Dataframes pré e pós balanceamento
corr = X_train.corr()
corr_random_under_sampler = pd.DataFrame(X_random_under_sampler).corr()

# criar os mapas de calor contendo os dados de treino pré e pós balanceamento
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (22,12))

sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, linewidths=.1, cmap="coolwarm", ax=ax[0])
sns.heatmap(corr_random_under_sampler, xticklabels=corr.columns, yticklabels=corr.columns, linewidths=.1, cmap="coolwarm", ax=ax[1])

ax[0].text(0, -0.5, 'Pré balanceamento',
         fontsize=16, 
         color="#000000",
         weight='bold')

ax[1].text(0, -0.5, 'Pós balanceamento',
         fontsize=16, 
         color="#000000",
         weight='bold')
plt.tight_layout()

Analisando os mapas de calor acima, notamos que após o balanceamento surgiram variáveis que possuem melhor relação entre elas.

## **Modelos de Machine Learning - Dados Balanceados**

Após o balanceamento de nosso *Dataframe*, iremos reconstruir os mesmos modelos de classificação para validar qual terá melhor performance em nossa previsão entre transações lícitas ou fraudulentas.

### Regressão Logística

In [ ]:
# instanciar o modelo
model_logistic_regression_balanced = LogisticRegression()

# treinamento do modelo
model_logistic_regression_balanced.fit(X_random_under_sampler, y_random_under_sampler)

# fazer previsões em cima de novos dados
y_pred_logistic_regression_balanced = model_logistic_regression_balanced.predict(X_test)

### Decision Trees

In [ ]:
# instanciar o modelo e escolher os hyperparameters
model_decision_tree_balanced = DecisionTreeClassifier(max_depth=4, criterion="entropy")

# treinamento do modelo
model_decision_tree_balanced.fit(X_random_under_sampler, y_random_under_sampler)

# fazer previsões em cima de novos dados
y_pred_decision_tree_balanced = model_decision_tree_balanced.predict(X_test)

### KNN

In [ ]:
# instanciar o modelo e escolher os hyperparameters
model_knn_balanced = neighbors.KNeighborsClassifier(n_neighbors=5)

# treinamento do modelo
model_knn_balanced.fit(X_random_under_sampler, y_random_under_sampler)

# fazer previsões em cima de novos dados
y_pred_knn_balanced = model_knn_balanced.predict(X_test)

## **Avaliar o desempenho dos modelos**

Nessa etapa iremos avaliar o desempenho dos 3 modelos utilizados para verificar qual deles será a melhor escolha para previsão de transações lícitas ou fraudulentas.

### Regressão Logística - Dados desbalanceados

In [ ]:
# importar a função classification_report
from sklearn.metrics import classification_report

# imprimir relatório de classificação
print(classification_report(y_test, y_pred_logistic_regression_unbalanced))

In [ ]:
# instalar skplt
!pip install -q scikit-plot

In [ ]:
# importar a função skplt
import scikitplot as skplt

# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_logistic_regression_unbalanced, normalize=True, cmap='Reds')
plt.show()

In [ ]:
# importar a função accuracy_score
from sklearn.metrics import accuracy_score

# imprimir a acurácia do modelo
result_accuracy_logistic_regression_unbalanced = accuracy_score(y_test, y_pred_logistic_regression_unbalanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_logistic_regression_unbalanced))

In [ ]:
# importar a função auc
from sklearn.metrics import roc_auc_score

# imprimir a área sob a curva
result_auc_logistic_regression_unbalanced = roc_auc_score(y_test, y_pred_logistic_regression_unbalanced)
print("AUC: {:.4f}\n".format(result_auc_logistic_regression_unbalanced))

### Regressão Logística - Dados balanceados

In [ ]:
# imprimir relatório de classificação
print(classification_report(y_test, y_pred_logistic_regression_balanced))

In [ ]:
# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_logistic_regression_balanced, normalize=True, cmap='Reds')
plt.show()

In [ ]:
# imprimir a acurácia do modelo
result_accuracy_logistic_regression_balanced = accuracy_score(y_test, y_pred_logistic_regression_balanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_logistic_regression_balanced))

In [ ]:
# imprimir a área sob a curva
result_auc_logistic_regression_balanced = roc_auc_score(y_test, y_pred_logistic_regression_balanced)
print("AUC: {:.4f}\n".format(result_auc_logistic_regression_balanced))

### Decision Trees - Dados desbalanceados

In [ ]:
# imprimir relatório de classificação
print(classification_report(y_test, y_pred_decision_tree_unbalanced))

In [ ]:
# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_decision_tree_unbalanced, normalize=True, cmap='Blues')
plt.show()

In [ ]:
# imprimir a acurácia do modelo
result_accuracy_decision_tree_unbalanced = accuracy_score(y_test, y_pred_decision_tree_unbalanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_decision_tree_unbalanced))

In [ ]:
# imprimir a área sob a curva
result_auc_decision_tree_unbalanced = roc_auc_score(y_test, y_pred_decision_tree_unbalanced)
print("AUC: {:.4f}\n".format(result_auc_decision_tree_unbalanced))

### Decision Trees - Dados balanceados

In [ ]:
# imprimir relatório de classificação
print(classification_report(y_test, y_pred_decision_tree_balanced))

In [ ]:
# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_decision_tree_balanced, normalize=True, cmap='Blues')
plt.show()

In [ ]:
# imprimir a acurácia do modelo
result_accuracy_decision_tree_balanced = accuracy_score(y_test, y_pred_decision_tree_balanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_decision_tree_balanced))

In [ ]:
# imprimir a área sob a curva
result_auc_decision_tree_balanced = roc_auc_score(y_test, y_pred_decision_tree_balanced)
print("AUC: {:.4f}\n".format(result_auc_decision_tree_balanced))

### KNN - Dados desbalanceados

In [ ]:
# imprimir relatório de classificação
print(classification_report(y_test, y_pred_knn_unbalanced))

In [ ]:
# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_knn_unbalanced, normalize=True, cmap='Oranges')
plt.show()

In [ ]:
# imprimir a acurácia do modelo
result_accuracy_knn_unbalanced = accuracy_score(y_test, y_pred_knn_unbalanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_knn_unbalanced))

In [ ]:
# imprimir a área sob a curva
result_auc_knn_unbalanced = roc_auc_score(y_test, y_pred_knn_unbalanced)
print("AUC: {:.4f}\n".format(result_auc_knn_unbalanced))

### KNN - Dados balanceados

In [69]:
# imprimir relatório de classificação
print(classification_report(y_test, y_pred_knn_balanced))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56847
           1       0.07      0.93      0.13       115

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.56     56962
weighted avg       1.00      0.97      0.99     56962



In [ ]:
# criar uma matriz de confusão
skplt.metrics.plot_confusion_matrix(y_test, y_pred_knn_balanced, normalize=True, cmap='Oranges')
plt.show()

In [ ]:
# imprimir a acurácia do modelo
result_accuracy_knn_balanced = accuracy_score(y_test, y_pred_knn_balanced)
print("Acurácia: {:.4f}\n".format(result_accuracy_knn_balanced))

In [ ]:
# imprimir a área sob a curva
result_auc_knn_balanced = roc_auc_score(y_test, y_pred_knn_balanced)
print("AUC: {:.4f}\n".format(result_auc_knn_balanced))

## **Conclusão**

Comparando os três modelos utilizados, podemos chegar no seguinte resultado:

In [ ]:
def format_percents(value):
  return "{:.4f}".format(value)

# criar um dicionário com dados para comparação
results = {'Modelos': ['Regressão Logística', 'Decision Trees', 'KNN'],
           'Acurácias desbalanceadas': [format_percents(result_accuracy_logistic_regression_unbalanced), format_percents(result_accuracy_decision_tree_unbalanced), format_percents(result_accuracy_knn_unbalanced)],
           'Acurácias balanceadas': [format_percents(result_accuracy_logistic_regression_balanced), format_percents(result_accuracy_decision_tree_balanced), format_percents(result_accuracy_knn_balanced)],
           'AUCs desbalanceadas': [format_percents(result_auc_logistic_regression_unbalanced), format_percents(result_auc_decision_tree_unbalanced), format_percents(result_auc_knn_unbalanced)],
           'AUCs balanceadas': [format_percents(result_auc_logistic_regression_balanced), format_percents(result_auc_decision_tree_balanced), format_percents(result_auc_knn_balanced)]}

# converter em DataFrame
compare = pd.DataFrame(data=results)

# importar a função data_table
from google.colab import data_table

# imprimir a tabela de comparação
data_table.DataTable(compare, include_index=False, num_rows_per_page=10)

Avaliando os três modelos propostos, podemos observar que as métricas de acurácia e AUC do modelo *KNN* foram maiores que os outros modelos avaliados.

Se comparamos os valores dos modelos desbalanceados e balanceados, podemos notar nos gráficos *Normalized Confusion Matrix* (ou na tabela *Classification Report, coluna recall*) anteriores que, quando utilizado os dados desbalanceados, as métricas para previsão de transações lícitas chegam a 100% de desempenho, mas para as transações fraudulentas temos um fraco desempenho. Já com os dados balanceados, temos um melhor desempenho tanto nas transações lícitas como nas fraudulentas. Sendo assim, notamos com essa comparação a suma importância que a etapa de balanceamento de dados tem sobre nossos modelos preditivos.

Podemos comparar também na tabela anterior que, as acurácias desbalanceadas são próximo de 100%, mas que a métrica AUC desbalanceada é bem inferior a métrica AUC balanceada, confirmando ainda mais nossa afirmação de que o balanceamento de dados é vital para o sucesso de nossos modelos preditivos.

Como citado no início desse projeto, a melhora desse algoritmo já proporciona uma segurança maior durante o uso de cartões de crédito, visto que fica mais preciso a sua eficácia.

Obrigado pela leitura e me acompanhe no [LinkedIn](https://www.linkedin.com/in/rodrigomariamorgao/) e no [Medium](https://medium.com/@rodrigomariamorgao) para outros artigos e análises. Abraço!